In [1]:
import musicbrainzngs
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
import requests
artist = "Chuck Berry"
response = requests.get(f"https://musicbrainz.org/ws/2/artist?query={artist}&fmt=json")
print(response.json())  # Primer resultado

{'error': 'Your requests are being throttled by MusicBrainz because the application you are using has not identified itself. Please update your application, and see http://musicbrainz.org/doc/XML_Web_Service/Rate_Limiting for more information.'}


In [33]:
import pandas as pd
import requests
import time
import re

# Leer el archivo de Spotify (ajusta el path si es necesario)
df_spotify = pd.read_csv("artists.csv")

artistas_raw = df_spotify['artist'].dropna().unique()
artistas = set()

# Función para separar artistas con múltiples delimitadores
def split_artists(texto):
    # Reemplazar "Featuring" por un delimitador temporal (e.g., ";") y limpiar
    texto = re.sub(r'\s*Featuring\s*', ',', texto, flags=re.IGNORECASE)
    # Separar por ";" y "&"
    partes = re.split(r'[,&]', texto)
    # Limpiar cada artista y devolver lista
    return [artista.strip() for artista in partes if artista.strip()]

# Procesar cada grupo de artistas
for grupo in artistas_raw:
    for artista in split_artists(grupo):
        artistas.add(artista)

print(f"🎵 Total de artistas únicos: {len(artistas)}")

🎵 Total de artistas únicos: 2967


In [40]:
import pandas as pd
import requests
import time


# Consultar MusicBrainz para cada artista
headers = {'User-Agent': 'ETLWorkshop/1.0 (tucorreo@dominio.com)'}
datos_artistas = []

for artist in list(artistas):  # puedes ajustar el número para pruebas
    query = artist.replace(" ", "+")
    url = f"https://musicbrainz.org/ws/2/artist/?query=artist:{query}&fmt=json&limit=1"
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        if data['artists']:
            a = data['artists'][0]
            datos_artistas.append({
                'artist_name': artist,
                'artist_id': a.get('id'),
                'country': a.get('country'),
                'begin_date': a.get('life-span', {}).get('begin'),
                'end_date': a.get('life-span', {}).get('end'),
                'type': a.get('type'),
                'gender': a.get('gender'),
                'disambiguation': a.get('disambiguation'),
            })
    else:
        print(f"❌ Error con {artist}: {response.status_code}")
    
    time.sleep(1)

# Guardar como CSV
df_api = pd.DataFrame(datos_artistas)

print("✅ Datos de MusicBrainz guardados en api_data.csv.")


✅ Datos de MusicBrainz guardados en api_data.csv.


In [42]:
df_api.to_csv("musicbrainz_artists.csv", index=False)

In [41]:
df_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2964 entries, 0 to 2963
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   artist_name     2964 non-null   object
 1   artist_id       2964 non-null   object
 2   country         2577 non-null   object
 3   begin_date      2532 non-null   object
 4   end_date        821 non-null    object
 5   type            2923 non-null   object
 6   gender          2016 non-null   object
 7   disambiguation  1189 non-null   object
dtypes: object(8)
memory usage: 185.4+ KB


In [43]:
df_api.head(50)

,artist_name,artist_id,country,begin_date,end_date,type,gender,disambiguation
0,Rhonda Vincent And The Rage,b6310fcc-30ea-41a9-9cda-1931f569d76f,US,None,None,Group,None,None
1,Yolanda Kondonassis,73e9c3b8-ef39-48e5-8b08-f746dcf0c765,US,1963,None,Person,female,harpist
2,John Hartford,3178cc4c-cd9f-4842-96fb-28bf9cfec591,US,1937-12-30,2001-06-04,Person,male,None
3,Sheryl Crow,80ccfede-c258-4575-a7ad-c982e9932e0f,US,1962-02-11,None,Person,female,American singer-songwriter and musician
4,Neal Hefti,75770f14-cbe0-4b06-8825-14797098ff2a,US,1922-10-29,2008-10-11,Person,male,None
5,Bayreuth Festival Orchestra,2914addc-dc71-41ce-a3b0-68d6a73bb5d0,None,1950,None,Orchestra,None,"“house orchestra” of Decca Records, associated..."
6,Thalma de Freitas With Vitor Gonçalves,dc31c385-c03e-4cec-acf4-450bbf877239,None,None,None,Person,male,None
7,Chicago Youth Symphony Orchestra,86a3e2f4-ff8a-4d53-9c89-b7c1e3c977ad,US,None,None,Orchestra,None,None
8,Creed,4b1a830b-0a1f-42e5-b8d5-1d6743912e99,US,1995,None,Group,None,"US rock band from Tallahassee, FL"
9,Taylor Swift,20244d07-534f-4eff-b4d4-930878889970,US,1989-12-13,None,Person,female,None


In [ ]:


# Información básica
print("=== Información del Dataset ===")
print(mb_df.info())
print("\n=== Primeras 5 filas ===")
print(mb_df.head())

# Valores nulos
print("\n=== Valores nulos por columna ===")
print(mb_df.isnull().sum())

# Análisis básico
print("\n=== Artistas por país ===")
print(mb_df['country'].value_counts().head(10))

# Visualizaciones
plt.figure(figsize=(12, 6))

# Distribución por país
plt.subplot(1, 2, 1)
sns.countplot(x='country', data=mb_df, order=mb_df['country'].value_counts().index[:10])
plt.title('Top 10 Países de Artistas')
plt.xticks(rotation=45)

# Tipos de artistas
plt.subplot(1, 2, 2)
sns.countplot(x='type', data=mb_df)
plt.title('Distribución por Tipo de Artista')

plt.tight_layout()
plt.show()